In [1]:
import pandas as pd
import numpy as np
import datetime as dt

Read and Clean Data 

In [2]:
df_trans = pd.read_csv("Trans_Nov.csv")
df_chart = pd.read_csv("Chart_Accounts.csv")

In [3]:
df_chart_new = df_chart.rename(columns={"Unnamed: 2":"Expense_Code", "40000":"Code"}) #clean chart of accounts
df_chart_new.dropna(how='all', inplace=True)#Clean or organize charts 
df_chart_new = df_chart_new[["Expense_Code", "Code"]]
df_chart_new.reset_index(drop=True, inplace = True)

In [4]:
df_chart_new.reset_index(drop=True, inplace = True) #df_chart_new is cleaned and ready

In [5]:
df_chart_new

,Expense_Code,Code
0,Donated Goods Retail,40000.0
1,Retail Sales - Rack Goods,40100.0
2,Retail Sales - Bin Goods,40200.0
3,Retail Sales - Furniture,40300.0
4,Retail Sales - E & M,40400.0
...,...,...
150,Cost of Compensation Allocation,79000.0
151,Allocated Indirect Admin Overhead,79100.0
152,Non Cash and Non Operating,90000.0
153,Depreciation & Amortization,90000.0


In [6]:
df_trans_new = df_trans[["Account", "Department.1", "Location.1", " Balance "]] #clean df_trans
df_trans_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4681 entries, 0 to 4680
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Account       4681 non-null   int64 
 1   Department.1  4681 non-null   object
 2   Location.1    3740 non-null   object
 3    Balance      4681 non-null   object
dtypes: int64(1), object(3)
memory usage: 146.4+ KB


In [ ]:
df_trans_new[' Balance '] = df_trans_new[' Balance '].str.replace("(","-")#make balance a float
df_trans_new[' Balance '] = df_trans_new[' Balance '].str.replace(')', "") #[' Balance '].str.replace(")","")
df_trans_new[" Balance "] = df_trans_new[" Balance "].str.replace(",","")

In [ ]:
df_trans_new[" Balance "] = df_trans_new[" Balance "].str.strip()
df_trans_new[" Balance "]= df_trans_new[" Balance "].astype(float)

In [ ]:
df_trans_new.insert(2, "Dept_Loc", value=2)

In [ ]:
df_trans_new["Location.1"] = df_trans_new["Location.1"].astype(str)

In [ ]:
#df_trans_new["Dept_Loc"].apply(lambda x:)
df_trans_new["Dept_Loc"] = df_trans_new["Department.1"] + "-"+df_trans_new["Location.1"]

In [ ]:
df_trans_final = df_trans_new.drop(columns=(["Department.1", "Location.1"]))

Create Grouping Lists

In [ ]:
salaries_list = [60220,60230,61010,61020,61030,61040,61100,61190,61310,61320,61330,61395,79000] #salaries, wage and benefits,taxes
client_wages_list = [60100,60210]
Admin = [79100]
Revenue = [43100,43200,43300,43400,43500,43600,43700,44600,44950]
comb_list = salaries_list + client_wages_list + Admin + Revenue
Departments = ['5657-100', '5657-200', '5658-100','5691-100', '5692-100', '5692-200', '5692-500', '5711-200', '5659-100']


Finding Total Expenses for Each Program and Catagory (S&B, Nonpersonnel, Admin, Client Wages, Revenue)

In [ ]:
wages = []
non_person = []
Administration = []
Client = []
R = []

for i in Departments:
    program = df_trans_final[df_trans_final["Dept_Loc"] == i] #Only OWP Data 

    sb_owp = program[program["Account"].isin(salaries_list)] #Provides just S&B expense codes
    elim_client = sb_owp[" Balance "].sum() #S&B total including client FICA and WC
    Cli_wages = program[program["Account"].isin(client_wages_list)] #Client Wages 
    wages.append(elim_client - (Cli_wages[" Balance "].sum())*.1675) #Eliminate Client FICA & WC from S&B

    non_personnel = program[~program["Account"].isin(comb_list)] #Non_personnel 
    non_person.append(non_personnel[" Balance "].sum())

    Administ = program[program["Account"].isin(Admin)] #Admin
    Administration.append(Administ[" Balance "].sum())

    Cli_wages = program[program["Account"].isin(client_wages_list)] #Client Wages & FICA
    Client.append((Cli_wages[" Balance "].sum())*1.1675)

    Rev = program[program["Account"].isin(Revenue)] #Total Revenue 
    R.append(Rev[" Balance "].sum())


In [ ]:
budget_components=[wages, R, Client, Administration, non_person] #Convert all lists to Series
wages = pd.Series(wages)
non_person = pd.Series(non_person)
Administration = pd.Series(Administration)
Client = pd.Series(Client)
R = pd.Series(R)
Departments = pd.Series(Departments)

In [ ]:
dashboard = pd.concat([Departments, wages, non_person, Administration, Client, R], axis=1)
dashboard.rename(columns={0:"DEPT", 1:"S&B", 2:"NON_PERSONNEL",3:"ADMINISTRATION",4:"CLIENT", 5:"REVENUE"}, inplace=True)
dashboard

Budget DataFrame 

In [ ]:
budget = {'S&B_Budget': [565000,337792,0,0,0,0,0,0,0], 'Non_Personnel_Budget': [62000,48893,0,0,0,0,0,0,0], 
          'Admin_Budget':[62700,38668,0,0,0,0,0,0,0], "Client_Budget":[710296,151841,0,0,0,0,0,0,0]}
df_budget = pd.DataFrame(budget)

In [ ]:
df_budget

In [ ]:
final_df = pd.concat([dashboard,df_budget], axis =1)

In [ ]:
final_df

In [ ]:
start_date = pd.to_datetime('7/1/2021')
end_date = pd.to_datetime(input("Input end date"))

In [ ]:
start_date

In [ ]:
end_date

In [ ]:
weeks = (end_date - start_date).days/7 #convert to weeks 
weeks

In [ ]:
final_df.assign(SB_Projection=lambda x:(final_df["S&B"]+(31*10900)))   #look into making two different ways to project

In [ ]:
final_df.info()